In [3]:
%%capture
!pip install openrouteservice
!pip install geopy
!pip install folium
!pip install ipywidgets

In [11]:
import openrouteservice
import ipywidgets as widgets
from IPython.display import display
import folium
from math import radians, sin, cos, sqrt, atan2
import numpy as np

ORS_API_KEY = "5b3ce3597851110001cf6248e69218e7578143a6a3b6711a43b4e590"
client = openrouteservice.Client(key=ORS_API_KEY)

CAR_CO2_PER_MILE = 404
BART_CO2_PER_MILE = 53
BUS_CO2_PER_MILE = 299

station_coords = {
    "Downtown Berkeley": (-122.2727, 37.8699),
    "16th St Mission": (-122.4190, 37.7652),
    "Embarcadero": (-122.3969, 37.7929),
    "Fremont": (-121.9764, 37.5573),
    "MacArthur": (-122.2672, 37.8291),
    "SFO": (-122.3929, 37.6159),
    "Richmond": (-122.3532, 37.9369),
    "El Cerrito Plaza": (-122.2993, 37.9026),
    "El Cerrito del Norte": (-122.3033, 37.9210),
    "North Berkeley": (-122.2778, 37.8747),
    "Ashby": (-122.2692, 37.8286),
    "19th St Oakland": (-122.2759, 37.8055),
    "12th St Oakland City Center": (-122.2711, 37.8044),
    "Lake Merritt": (-122.2670, 37.7992),
    "Fruitvale": (-122.2339, 37.7751),
    "Coliseum": (-122.2106, 37.7526),
    "San Leandro": (-122.1607, 37.7241),
    "Bay Fair": (-122.1212, 37.6877),
    "Hayward": (-122.0803, 37.6688),
    "South Hayward": (-122.0422, 37.6523),
    "Union City": (-121.9821, 37.5955),
    "Pittsburg/Bay Point": (-121.9983, 38.0165),
    "North Concord/Martinez": (-122.0276, 38.0036),
    "Concord": (-122.0295, 38.0133),
    "Pleasant Hill": (-122.0346, 37.9357),
    "Walnut Creek": (-122.0658, 37.9021),
    "Lafayette": (-122.1184, 37.8860),
    "Orinda": (-122.1823, 37.8766),
    "Rockridge": (-122.2537, 37.8293),
    "West Oakland": (-122.2953, 37.8054),
    "Montgomery": (-122.4017, 37.7833),
    "Powell": (-122.4076, 37.7839),
    "Civic Center": (-122.4157, 37.7795),
    "24th St Mission": (-122.4195, 37.7527),
    "Glen Park": (-122.4349, 37.7254),
    "Balboa Park": (-122.4485, 37.7246),
    "Daly City": (-122.4731, 37.7103),
    "Colma": (-122.4691, 37.6849),
    "South SF": (-122.4465, 37.6511),
    "San Bruno": (-122.4215, 37.6349),
    "Millbrae": (-122.3942, 37.6062)
}

bart_lines = [
    ['Richmond', 'El Cerrito del Norte', 'El Cerrito Plaza', 'North Berkeley', 'Downtown Berkeley',
     'Ashby', 'MacArthur', '19th St Oakland', '12th St Oakland City Center', 'Lake Merritt', 'Fruitvale',
     'Coliseum', 'San Leandro', 'Bay Fair', 'Hayward', 'South Hayward', 'Union City', 'Fremont'],
    ['Pittsburg/Bay Point', 'North Concord/Martinez', 'Concord', 'Pleasant Hill', 'Walnut Creek',
     'Lafayette', 'Orinda', 'Rockridge', 'MacArthur', '19th St Oakland', '12th St Oakland City Center',
     'West Oakland', 'Embarcadero', 'Montgomery', 'Powell', 'Civic Center', '16th St Mission', '24th St Mission',
     'Glen Park', 'Balboa Park', 'Daly City', 'Colma', 'South SF', 'San Bruno', 'SFO', 'Millbrae'],
    ['Richmond', 'El Cerrito del Norte', 'El Cerrito Plaza', 'North Berkeley', 'Downtown Berkeley', 'Ashby',
     'MacArthur', '19th St Oakland', '12th St Oakland City Center', 'West Oakland', 'Embarcadero', 'Montgomery',
     'Powell', 'Civic Center', '16th St Mission', '24th St Mission', 'Glen Park', 'Balboa Park', 'Daly City',
     'Colma', 'South SF', 'San Bruno', 'SFO', 'Millbrae']
]

def haversine(coord1, coord2):
    R = 3958.8  # miles
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

def get_bart_distance(start, end):
    for line in bart_lines:
        if start in line and end in line:
            start_idx = line.index(start)
            end_idx = line.index(end)
            if start_idx > end_idx:
                start_idx, end_idx = end_idx, start_idx
            distance = 0
            for i in range(start_idx, end_idx):
                distance += haversine(station_coords[line[i]], station_coords[line[i + 1]])
            return round(distance, 2)
    print('No direct route exists between these stations. Please select two stations on the same line.')

def get_route_distance(start_coord, end_coord):
    try:
        route = client.directions(
            coordinates=[start_coord, end_coord],
            profile='driving-car',
            format='geojson'
        )
        distance_m = route['features'][0]['properties']['segments'][0]['distance']
        return distance_m / 1609.34
    except Exception as e:
        return None

stations = list(station_coords.keys())

title = widgets.HTML("<h2>BART Emissions Calculator</h2>")
description = widgets.HTML("<p>Enter the start and end stations to calculate emissions for car, bus, and bart routes between the two stations.</p>")

start_dropdown = widgets.Dropdown(options=stations, description="Start:")
end_dropdown = widgets.Dropdown(options=stations, description="End:")
button = widgets.Button(description="Compare CO₂")
output = widgets.Output()

display(title, description, start_dropdown, end_dropdown, button, output)

def get_all_emissions(start, end):
    start_coord = station_coords[start]
    end_coord = station_coords[end]
    drive_miles = get_route_distance(start_coord, end_coord)
    if drive_miles is None:
        print("Could not get route distance.")
        return
    bart_miles = get_bart_distance(start, end)
    car_kg = drive_miles * CAR_CO2_PER_MILE / 1000
    bart_kg = bart_miles * BART_CO2_PER_MILE / 1000
    bus_kg = drive_miles * BUS_CO2_PER_MILE / 1000
    return car_kg, bus_kg, bart_kg

def compare_emissions(b):
    with output:
        output.clear_output()
        start = start_dropdown.value
        end = end_dropdown.value
        
        if start == end:
            print("Please select two different stations.")
            return
        
        start_coord = station_coords[start]
        end_coord = station_coords[end]

        drive_miles = get_route_distance(start_coord, end_coord)
        if drive_miles is None:
            print("Could not get route distance.")
            return
        bart_miles = get_bart_distance(start, end)
        if not bart_miles:
            return None
        car_kg = drive_miles * CAR_CO2_PER_MILE / 1000
        bart_kg = bart_miles * BART_CO2_PER_MILE / 1000
        bus_kg = drive_miles * BUS_CO2_PER_MILE / 1000
        print(f"From {start} to {end}:")
        print(f"Driving/bus distance: {drive_miles:.2f} miles")
        print(f"BART distance:  {bart_miles:.2f} miles")
        print(f"Car CO₂:  {car_kg:.2f} kg")
        print(f"Bus CO₂: {bus_kg:.2f} kg")
        print(f"BART CO₂: {bart_kg:.2f} kg")
        print(f"CO₂ difference bart/car: {(car_kg - bart_kg)/bart_kg*100:.2f}%")
        print(f"CO₂ difference bus/car: {(car_kg - bus_kg)/bus_kg*100:.2f}%")

        m = folium.Map((np.array(start_coord)[::-1] + np.array(end_coord)[::-1])/2, zoom_start=10)
        folium.Marker(list(start_coord)[::-1], popup=start).add_to(m)
        folium.Marker(list(end_coord)[::-1], popup=end).add_to(m)
        display(m)

button.on_click(compare_emissions)

HTML(value='<h2>BART Emissions Calculator</h2>')

HTML(value='<p>Enter the start and end stations to calculate emissions for car, bus, and bart routes between t…

Dropdown(description='Start:', options=('Downtown Berkeley', '16th St Mission', 'Embarcadero', 'Fremont', 'Mac…

Dropdown(description='End:', options=('Downtown Berkeley', '16th St Mission', 'Embarcadero', 'Fremont', 'MacAr…

Button(description='Compare CO₂', style=ButtonStyle())

Output()